- take input sequences and make blast database
- create pairwise combos by index

Alternatively:
- take pairwise sequences
- make blast database with unique seqs

Now have blast database and pairwise sequences (need to assign indices)

- Run Blast via iterator?
- Generate feature set for model
- Send to model

In [1]:
import os
import sys
import re
import shutil
import pandas as pd
import numpy as np
import tempfile

from typing import Collection

from itertools import combinations
import io

from Bio.Blast.Applications import NcbimakeblastdbCommandline, NcbiblastpCommandline
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# https://biopython.org/docs/1.75/api/Bio.pairwise2.html
from Bio import pairwise2
from Bio.Align import substitution_matrices

/home/ryfran/miniconda3/envs/validprot/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
s50k = pd.read_csv('../learn2therm_sample_50k_exploration.csv')

In [3]:
s50k['thermo_index'][0]

875

In [4]:
pd.set_option('display.max_columns', 40)
s50k.describe()

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,meso_index,prot_pair_index,meso_protein_int_index,thermo_protein_int_index,taxa_pair_index,local_gap_compressed_percent_id_16s,scaled_local_query_percent_id_16s,scaled_local_symmetric_percent_id_16s,query_align_cov_16s,subject_align_cov_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_len,t_protein_len
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,24999.500000,0.349026,0.276478,0.270792,206.011660,0.791507,205.414920,0.770875,230.63206,8296.524160,8098.858120,9.105934e+07,3.213584e+07,2.600952e+07,362827.905140,0.906828,0.903266,0.904959,0.997153,0.997756,1061.404480,27.348365,51.505410,24.157045,269.985200,261.367700
std,14433.901067,0.096608,0.114404,0.116039,64.260901,0.144893,64.430548,0.179955,186.44859,5181.817228,4608.982941,5.233883e+07,1.868143e+07,1.662125e+07,229264.926253,0.015240,0.015626,0.015300,0.007108,0.003933,75.659661,3.737201,3.816972,3.586797,65.935753,69.675548
min,0.000000,0.164103,0.081301,0.045977,19.000000,0.284553,19.000000,0.082019,35.00000,37.000000,5.000000,1.219000e+03,8.554400e+04,3.063480e+05,4885.000000,0.876590,0.836055,0.880026,0.871051,0.866066,891.000000,7.000000,41.000000,20.000000,30.000000,27.000000
25%,12499.750000,0.293750,0.204545,0.202020,172.000000,0.673288,172.000000,0.656854,118.00000,3841.000000,4121.000000,4.645832e+07,1.619813e+07,1.628730e+07,173976.750000,0.895995,0.893020,0.893531,0.996728,0.996719,1009.000000,27.000000,50.000000,22.000000,233.000000,226.000000
50%,24999.500000,0.325000,0.250000,0.244648,210.000000,0.811321,209.000000,0.806584,186.00000,7134.000000,8046.000000,9.160212e+07,3.248324e+07,2.353333e+07,312362.000000,0.905983,0.902326,0.903543,0.997978,0.998009,1056.000000,28.000000,52.500000,24.500000,267.000000,260.000000
75%,37499.250000,0.367257,0.308300,0.301639,240.000000,0.923077,240.000000,0.923077,267.00000,14062.000000,12103.000000,1.360713e+08,4.819069e+07,2.868282e+07,617814.000000,0.916172,0.912576,0.914698,0.999350,0.999350,1108.000000,29.000000,52.500000,24.500000,317.000000,311.000000
max,49999.000000,0.985075,0.985075,0.985075,400.000000,1.000000,399.000000,1.000000,1788.00000,16507.000000,16564.000000,1.814866e+08,6.542306e+07,6.476579e+07,744518.000000,0.999351,0.993544,0.996439,1.000000,1.000000,1679.000000,40.000000,78.000000,56.000000,400.000000,400.000000


In [5]:
s50k['m_protein_seq'][:10].to_csv('single_list_test.csv', index = False)
s50k[['m_protein_seq', 't_protein_seq']][:10].to_csv('pair_list_test.csv', index = False)

In [6]:
slist = pd.read_csv('single_list_test.csv')

In [7]:
class FafsaPairs:
    
    def __init__(self, path):
        
        self.pairs = np.array(pd.read_csv(path))
        
    def align(self):
        
    
        

        
        
# Class for single sequence list input. Inherits from FAFSA_paired but is able to parse a single column input.
class FafsaSingles(FafsaPairs):
    
    def __init__(self, path):
        
        self.pairs = list(combinations(pd.read_csv(path).values, 2))


IndentationError: expected an indented block after function definition on line 7 (60237676.py, line 15)

In [34]:
mesos = pd.read_csv('s5.csv')[['m_protein_seq', 'meso_index']]

In [42]:
test = open('test.fasta','w')

for row in mesos.iterrows():
    test.write(f">{row[1]['meso_index']}\n{row[1]['m_protein_seq']}\n")
test.close()

In [44]:
cmd = NcbimakeblastdbCommandline(dbtype='prot', input_file='test.fasta', parse_seqids=True)()


In [45]:
NcbiblastpCommandline(query='test_q.fasta', db='test.fasta', outfmt=5, out='test_out', word_size=3, evalue=10000, max_target_seqs=100000)()


('', '')

In [170]:
class BlastMet:
    """Handles computation of metrics for each alignment in a blast record.

    The HSP with the largest average sequence coverage is used for local metrics.
    
    Parameters
    ----------
    blast_record : the record containing all hits for a query
    """
    def __init__(self, blast_record):
        recs = []
        for rec in blast_record:
            recs.append(rec)           # This is clunky
        self.record = recs[0]
        self.qid = self.record.query.split(' ')[0]

    def id_hsp_best_cov(self, alignment):
        """Determine HSP with the most average coverage of both sequences.
        
        Returns
        -------
        Index of HSP with max average seq coverage
        Max average coverage
        """
        scores = []

        for hsp in alignment.hsps:
            scores.append(
                ((hsp.query_end +1 - hsp.query_start)/self.record.query_length + (hsp.sbjct_end +1 - hsp.sbjct_start)/
                 alignment.length)/2)
        return np.argmax(scores), max(scores)        
    
    @staticmethod
    def raw_gap_compressed_percent_id(n_matches, n_gaps, n_columns, n_compressed_gaps):
        """Percent matches in sequence, including but compressing gaps.
        
        Parameters
        ----------
        n_matches : int, number of matches in match columns
        n_gaps : number of gaps in match columns
        n_columns : total number of alignment match columns
        n_compressed_gaps : number of compressed gaps in match columns
        """
        return n_matches / (n_columns - n_gaps + n_compressed_gaps)
    
    def compute_metric(self,metric_name: str):
        """Compute the metric with specified name for each alignment"""
        if not hasattr(self, metric_name):
            raise ValueError(f"No metric found with name : {metric_name}")
        else:
            metric = getattr(self, metric_name)

        outputs = []
        for alignment in self.record.alignments:
            hsp_id, _ = self.id_hsp_best_cov(alignment)
            hsp = alignment.hsps[hsp_id]
            outputs.append((self.qid, alignment.hit_id.split('|')[-1], metric(alignment, hsp)))
        return pd.DataFrame(data=outputs, columns=['query_id', 'subject_id', metric_name])
    
    def ids(self, alignment, hsp):
            return 0

    def subject_align_cov(self, alignment, hsp):
        """Fraction of AA on query string taken up by alignment"""
        return (hsp.sbjct_end +1 - hsp.sbjct_start)/alignment.length
    
    def subject_align_len(self, alignment, hsp):
        """Length of AA on query string taken up by alignment"""
        return int(hsp.sbjct_end +1 - hsp.sbjct_start)

    def query_align_cov(self, alignment, hsp):
        """Fraction of AA on query string taken up by alignment"""
        return (hsp.query_end +1 - hsp.query_start)/self.record.query_length

    def query_align_len(self, alignment, hsp):
        """Length of AA on query string taken up by alignment"""
        return int(hsp.query_end +1 - hsp.query_start)

    def bit_score(self, alignment, hsp):
        return hsp.score

    def local_gap_compressed_percent_id(self, alignment, hsp):
        """Percent matches in match sequence, including but compressing gaps.
        
        The largest local HSP score is used
        """
        n_matches = hsp.identities
        n_gaps = hsp.gaps
        n_columns = len(hsp.query)
        n_compressed_gaps = len(re.findall('-+', hsp.query))+len(re.findall('-+', hsp.sbjct))
        return self.raw_gap_compressed_percent_id(n_matches, n_gaps, n_columns, n_compressed_gaps)

    def scaled_local_query_percent_id(self, alignment, hsp):
        """Percent matches in query sequence based on best HSP."""
        return hsp.identities/self.record.query_length

    def scaled_local_symmetric_percent_id(self, alignment, hsp):
        """Percent matches compared to average seq length of query and subject based on best HSP"""
        return 2*hsp.identities/(self.record.query_length + alignment.length)

In [54]:
result_handle = open('test_out')
blast_records = NCBIXML.parse(result_handle)
test = BlastMet(blast_records)

m = [
    'local_gap_compressed_percent_id',
    'scaled_local_query_percent_id',
    'scaled_local_symmetric_percent_id',
    'query_align_len',
    'query_align_cov',
    'subject_align_len',
    'subject_align_cov',
    'bit_score'
]

outdf = test.compute_metric('ids')[['query_id','subject_id']]

for metric in m: 
    outdf[metric] = test.compute_metric(metric)[metric]
    
outdf.head()

,query_id,subject_id,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score
0,875,6118,0.5,0.069307,0.059574,27,0.133663,27,0.100746,24.0


In [105]:
test100 = s50k.sample(10)[['m_protein_seq', 't_protein_seq']]
test100.to_csv('test100.csv')

In [86]:
data.head()

,query,subject,qid,sid,pid
0,MSDEIQVEGEELPHLLLVDDDATFTRVMARAMSRRGFRVSTAGSAE...,MRVLIIDDHALFRVGLQELLERRGIEVVGAVGDGVEGMELAERLRP...,65,67,0
1,MLERLRFIPSRLLHAIPVVFGVTILVFFMARMLPGDPATALLGEYA...,MLSYVGRRVLLMIPTVFVISVVAFAIIQLPPGDYLTTIVSQLQAQG...,35,43,1
2,MDVPSPSPDVLHLADVVPAVLAAMGVTDAQREGVHHHIDLPDDIRG...,MPDESTQSIVVDAPADRVMAVIADLESYPEWAKAVRRTEVLERDDH...,12,50,2
3,MTDLNEKVAIVTGASKGVGAAIARRFVESGARVIAAARSRDKLEAL...,MPQTSRTVIVTGAARGIGYAIAEAFAREGDRVIVADLDGDGADQAA...,81,54,3
4,MTTTQLDQHQPGSVVAARATGLTKTYGHGESAVHALRDVDLALLEG...,MGVEIRVEGLTKTFGRQVIWQDVSLTLPAGEISVLLGPSGTGKSVF...,91,27,4


In [132]:
data = pd.read_csv('test100.csv')[['m_protein_seq','t_protein_seq']]

In [134]:
original_cols = data.columns

data.rename(columns = {original_cols[0]: 'query', original_cols[1]: 'subject'}, inplace = True)

n_unique_1 = np.unique(data['query'])
n_unique_2 = np.unique(data['subject'])

qid_dict = dict(zip(n_unique_1, range(len(n_unique_1))))
sid_dict = dict(zip(n_unique_2, range(len(n_unique_2))))

data['qid'] = [qid_dict[i] for i in data.iloc[:,0]]
data['sid'] = [sid_dict[i] for i in data.iloc[:,1]]

data['pid'] = data.index

In [136]:
m = [
    'local_gap_compressed_percent_id',
    'scaled_local_query_percent_id',
    'scaled_local_symmetric_percent_id',
    'query_align_len',
    'query_align_cov',
    'subject_align_len',
    'subject_align_cov',
    'bit_score'
]
final = pd.DataFrame()

db = open('db.fasta','w')
for row in data.iterrows():

    db = open('db.fasta','w')
    db.write(f">{row[1]['sid']}\n{row[1]['subject']}\n")
    db.close()
    
    query = open('query.fasta','w')
    query.write(f">{row[1]['qid']}\n{row[1]['query']}\n")
    query.close()

    cmd = NcbimakeblastdbCommandline(dbtype='prot', input_file='db.fasta', parse_seqids=True)()
    NcbiblastpCommandline(query='query.fasta', db='db.fasta', outfmt=5, out='test_out', word_size=3, evalue=10000, max_target_seqs=100000)()
    
    result_handle = open('test_out')
    blast_records = NCBIXML.parse(result_handle)
    test = BlastMet(blast_records)

    outdf = test.compute_metric('ids')[['query_id','subject_id']]

    for metric in m: 
        outdf[metric] = test.compute_metric(metric)[metric]
    final = pd.concat([final, outdf], ignore_index = True)
    

In [121]:
final.shape

(10, 10)

In [101]:
final = final.astype({"query_id": int, "subject_id": int})

In [102]:
data.merge(final, left_on = ['qid','sid'], right_on = ['query_id','subject_id'])

,query,subject,qid,sid,pid,query_id,subject_id,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score
0,MSDEIQVEGEELPHLLLVDDDATFTRVMARAMSRRGFRVSTAGSAE...,MRVLIIDDHALFRVGLQELLERRGIEVVGAVGDGVEGMELAERLRP...,65,67,0,65,67,0.360360,0.215054,0.197044,110,0.591398,111,0.504545,143.0
1,MLERLRFIPSRLLHAIPVVFGVTILVFFMARMLPGDPATALLGEYA...,MLSYVGRRVLLMIPTVFVISVVAFAIIQLPPGDYLTTIVSQLQAQG...,35,43,1,35,43,0.306962,0.305031,0.299383,312,0.981132,329,0.996970,340.0
2,MDVPSPSPDVLHLADVVPAVLAAMGVTDAQREGVHHHIDLPDDIRG...,MPDESTQSIVVDAPADRVMAVIADLESYPEWAKAVRRTEVLERDDH...,12,50,2,12,50,0.347826,0.020888,0.030189,23,0.060052,23,0.156463,29.0
3,MTDLNEKVAIVTGASKGVGAAIARRFVESGARVIAAARSRDKLEAL...,MPQTSRTVIVTGAARGIGYAIAEAFAREGDRVIVADLDGDGADQAA...,81,54,3,81,54,0.352332,0.293103,0.278689,190,0.818966,195,0.761719,264.0
4,MTTTQLDQHQPGSVVAARATGLTKTYGHGESAVHALRDVDLALLEG...,MGVEIRVEGLTKTFGRQVIWQDVSLTLPAGEISVLLGPSGTGKSVF...,91,27,4,91,27,0.322115,0.264822,0.214744,211,0.833992,209,0.563342,226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,MNPMTVAVKAGFRRGAIELRQSFSNGMELFTQFFWPLLTLVALFFM...,MSAVLVQAGNLTLRELRALARQPFYVAVSLVQPLIWLLLFGQLFTS...,43,72,95,43,72,0.278146,0.148410,0.155268,158,0.558304,157,0.608527,55.0
96,MRLDNQLVESLRFARADDGVTIGYQMFGQGPVLVWMPSLSNIVAQW...,MIDELTVPGTPELVVADFGGRPDGPAVVLLHGFGGNCLHWAGFHWA...,59,30,96,59,30,0.325000,0.037037,0.039574,40,0.113960,40,0.130719,54.0
97,MKQRLEKLQTWMKENEIEVSFLTSSENVFYLSGYYTDPHERLLALV...,MSLIKNRKQIEAMHRAAKILAACHQEIGKGIRPGVTTLEIDRFVED...,32,76,97,32,76,0.298643,0.180822,0.215334,217,0.594521,226,0.911290,203.0
98,MDGPYLRLEESGDVRPLSGEVTTVGRGEGADIRLHDPSVSQLHAEL...,MRPERVVLAEDGVLLREGLAGLLERFGFEVVAAVGDAPALLDAVAE...,8,62,98,8,62,0.363014,0.263682,0.251781,157,0.781095,145,0.659091,70.0


In [100]:
final['query_id'][0]

'65'

In [171]:
def make_blast_df(paired_df):
    
    original_cols = data.columns

    data.rename(columns = {original_cols[0]: 'query', original_cols[1]: 'subject'}, inplace = True)

    n_unique_1 = np.unique(data['query'])
    n_unique_2 = np.unique(data['subject'])

    qid_dict = dict(zip(n_unique_1, range(len(n_unique_1))))
    sid_dict = dict(zip(n_unique_2, range(len(n_unique_2))))

    data['qid'] = [qid_dict[i] for i in data.iloc[:,0]]
    data['sid'] = [sid_dict[i] for i in data.iloc[:,1]]

    data['pid'] = data.index
    
    m = [
    'local_gap_compressed_percent_id',
    'scaled_local_query_percent_id',
    'scaled_local_symmetric_percent_id',
    'query_align_len',
    'query_align_cov',
    'subject_align_len',
    'subject_align_cov',
    'bit_score'
    ]
    
    final = pd.DataFrame()

    db = open('db.fasta','w')
    
    for row in data.iterrows():

        db = open('db.fasta','w')
        db.write(f">{row[1]['sid']}\n{row[1]['subject']}\n")
        db.close()

        query = open('query.fasta','w')
        query.write(f">{row[1]['qid']}\n{row[1]['query']}\n")
        query.close()

        cmd = NcbimakeblastdbCommandline(dbtype='prot', input_file='db.fasta', parse_seqids=True)()
        NcbiblastpCommandline(query='query.fasta', db='db.fasta', outfmt=5, out='test_out', word_size=3, evalue=10000, max_target_seqs=100000)()

        result_handle = open('test_out')

        blast_records = NCBIXML.parse(result_handle)

        test = BlastMet(blast_records)

        outdf = test.compute_metric('ids')[['query_id','subject_id']]

        for metric in m: 
            outdf[metric] = test.compute_metric(metric)[metric]
        final = pd.concat([final, outdf], ignore_index = True)
    
    final = final.astype({"query_id": int, "subject_id": int})
    
    blast_df = data.merge(final, left_on = ['qid','sid'], right_on = ['query_id','subject_id'])
    
    return(blast_df)

In [172]:
newdf = make_blast_df(pd.read_csv('test100.csv')[['m_protein_seq','t_protein_seq']])

In [169]:
out = blast2(newdf)

IndexError: list index out of range

In [166]:
blast_records = NCBIXML.parse(open('test_out'))
test = BlastMet(blast_records)


In [161]:
result_handle

<_io.TextIOWrapper name='test_out' mode='r' encoding='UTF-8'>